In [1]:
import numpy as np
import matplotlib.pyplot as plt

class LTI_Continuous:
    def __init__(self, impulse_response):
        self.impulse_response = impulse_response
        self.INF = 1000
        
    def linear_combination_of_impulses(self, input_signal, delta):
        decomposed = []
        
        # Use np.arange to allow non-integer delta
        for i in np.arange(-self.INF, self.INF + 1, delta):
            coefficient = input_signal.get_value_at_time(i)
            
            if coefficient != 0:
                # Create unit impulse
                impulse = ContinuousSignal(unit_impulse)
                
                # Shift impulse by i
                shifted_impulse = impulse.shift_signal(i)
                                
                # Append the shifted impulse and its coefficient
                decomposed.append((shifted_impulse, coefficient, i))

        return decomposed
    
    def output_approx(self, signal, delta):
        # Create a new continuous signal with all values set to 0
        result = ContinuousSignal(lambda x: 0)
        
        # Decompose the input signal
        decomposed = self.linear_combination_of_impulses(signal, delta)
        
        # For each decomposed signal, compute the output
        for decomposed_signal, coefficient, i in decomposed:
            shifted_response = self.impulse_response.shift_signal(i)
            component = shifted_response.multiply_const_factor(coefficient)
            result = result.add(component)
        
        return result 
    
    def show(self):
        self.impulse_response.plot(-10, 10, 'Impulse Response')


# Example usage
def unit_impulse(x):
    delta = 0.5
    impulse = np.zeros_like(x)
    impulse[(x >= 0) & (x < delta)] = 1 / delta
    return impulse

# ContinuousSignal class (same as previous with vectorization fix)
class ContinuousSignal:
    def __init__(self, func):
        self.func = func
    
    def get_value_at_time(self, time):
        return self.func(time)
    
    def add(self, other):
        return ContinuousSignal(lambda x: np.vectorize(self.func)(x) + np.vectorize(other.func)(x))
    
    def shift_signal(self, shift):
        return ContinuousSignal(lambda x: np.vectorize(self.func)(x - shift))
    
    def multiply(self, other):
        return ContinuousSignal(lambda x: np.vectorize(self.func)(x) * np.vectorize(other.func)(x))
    
    def multiply_const_factor(self, scaler):
        return ContinuousSignal(lambda x: np.vectorize(self.func)(x) * scaler)
    
    def plot(self, start, end, label):
        x = np.linspace(start, end, 1000)
        y = self.func(x)
        plt.plot(x, y, label=label)
        plt.xlabel('Time')
        plt.ylabel('Value')
        plt.grid(True)
    
    @staticmethod
    def show():
        plt.legend()
        plt.show()


# Define input and impulse response signals
def myFunc(x):
    return np.sin(x)

signal = ContinuousSignal(myFunc)
response = ContinuousSignal(unit_impulse)

# Initialize the system
system = LTI_Continuous(response)

# Compute the output with non-integer delta
output = system.output_approx(signal, 0.5)

# Plot the output
output.plot(-10, 10, 'Output')
ContinuousSignal.show()


RecursionError: maximum recursion depth exceeded in comparison